In [ ]:
import numpy as np
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = sigmoid(a2)
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        return mean_squared_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = (y > 0.5).astype(int)
        accuracy = np.mean(y == t)
        return accuracy
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1 
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = sigmoid(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads

np.random.seed(42) 
train_size = 100
test_size = 20

Ftrain_x = np.random.normal(160, 5, (train_size//2, 1))
Ftest_x = np.random.normal(160, 5, (test_size//2, 1))
Mtrain_x = np.random.normal(173, 5, (train_size//2, 1))
Mtest_x = np.random.normal(173, 5, (test_size//2, 1))

train_x = np.vstack((Ftrain_x, Mtrain_x))
train_t = np.vstack((np.zeros((train_size//2, 1)), np.ones((train_size//2, 1))))
test_x = np.vstack((Ftest_x, Mtest_x))
test_t = np.vstack((np.zeros((test_size//2, 1)), np.ones((test_size//2, 1))))

mean = np.mean(train_x)
std = np.std(train_x)
train_x = (train_x - mean) / std
test_x = (test_x - mean) / std

network = TwoLayerNet(1, 100, 1)
iters_num = 10000
learning_rate = 0.001
iter_per_epoch = 100

for i in range(iters_num):
    grad = network.gradient(train_x, train_t)
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(train_x, train_t)
        test_acc = network.accuracy(test_x, test_t)
        print(i/100, f" train acc: {train_acc:.2f}, test acc: {test_acc:.2f}")